<div style="text-align: right"><strong>Capstone #3:</strong> <span style="color:darkred">Supervised Learning</span> </div>

<a id="top"></a>

#### <span style="color:darkred">__Part 1: Data Exploration__ https://github.com/kimrharper/thinkful/blob/master/unit3/unit3-capstone-exploration.ipynb </span><br><br><span style="color:darkred">__Part 2: Models__ https://github.com/kimrharper/thinkful/blob/master/unit3/unit3-capstone-models.ipynb </span>

----

# <span style="color:darkred">Part 2: </span><span style="color:darkblue">L1 Prediction from ELL Writing Samples</span>

__Author:__ Ryan Harper 

----

<a href='#ov'>Overview</a><br>
<a href='#exp'>Experiment</a><br>
<a href='#sec1'>1. Models:</a><br>
><a href='#seca'>A. LR - Ordinary Least Squares</a><br>
<a href='#secb'>B. LR - Logistic Regression</a> <a href='#secb1'> (Lasso)</a> <a href='#secb2'> (Ridge)</a><br>
<a href='#secc'>C. NN - K Nearest Neighbors</a><br>
<a href='#secd'>D. NN - Naive Bayes</a><br>
<a href='#sece'>E. NN - Decision Tree</a><br>
<a href='#secf'>F. Ensemble - Random Forest</a><br>

<a href='#sec2'>2. Model Comparison</a><br>

<a id="ov"></a>

# <span style="color:darkblue">Overview</span>  <a href='#top'>(top)</a>

__Data Source:__
> http://lang-8.com/ [scraped with Beautiful Soup]

![alt text](../data/language/lang8.png "Title")

__Summary:__
> In my previous profession, I have been teaching English to a diverse range of students of all ages, language background, and country origin. During my professional development, I started to observe that different students with different L1s (1st Language) tended to display different patterns of communication that appeared to have some connection to either education in their country of origin or a connection to the linguistic structure of their first language. Different ELL (English Language Learners) needed to focus on different aspects of the English language depending on their background. The purpose of this project is to use a large number of blog posts from a language practicing website and explore whether or not the L1 has any significant impact on the blog writing style of the English learner.<br><br>Part 1: Explore the data to find any noteworthy trends in linguistic structure: <ol><li> vocabulary (word freq, collocations, and cognates) <li>syntax (sentence structure)<li>grammar (i.e. grammar complexity of sentences) <li>errors (types of errors) <li> parts of speech (NLTK Abbreviations: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/)</ol><br>Part 2: Use linguistic trends to determine whether or not a learner's first language can be predicted.

__Variables:__
>__id:__ _User ID_<br>
__time:__ _Time the blog post was scraped (in order of user posted time)_ <br>
__title:__ _Title of the blog post_<br>
__content:__ _The blog post_<br>
__language:__ _User's self-reported first language_

<a id="exp"></a>

# <span style="color:darkblue">Experiment</span> <a href='#top'>(top)</a>

__Hypothesis:__ 
> L1 (first language) experience and academic environment influences ELLs' (English Language Learners') writing style. The L1 of ELLs can be predicted by looking at English blog posts and identifying patterns unique to their L1.

__Observations:__
><li> --<li>--<li>--

__Method:__
> Using multiple different models. The aim of this project is to explore how different models can handle the data (target and features) and to see what information can be gained from using multiple different models. Ultimately, the goal is to determine which models are appropriate for a binary (discrete) target with features that are both qualitative (discrete) and quantitative (ranked/continuous).

<a id="sec1"></a>

# <span style="color:darkblue">1. Models:</span>  <a href='#top'>(top)</a>

In [1]:
# iPython/Jupyter Notebook
import time
from pprint import pprint
import warnings
from IPython.display import Image

import time

# Data processing
import pandas as pd
import plotly as plo
import seaborn as sns
from scipy import stats
from collections import Counter
import numpy as np
import itertools

# NLP
from nltk.corpus import stopwords as sw
from nltk.util import ngrams
from nltk.corpus import brown
import nltk
import re
from nltk.tokenize import RegexpTokenizer
import difflib

# Stats
from sklearn.metrics import classification_report, roc_curve,roc_auc_score,accuracy_score
from sklearn import metrics

# Preparing Models
from sklearn.model_selection import train_test_split

# Models
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB

# Ensemble
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

#Visualization
from IPython.display import Image
import pydotplus
import graphviz

# import altair as alt

In [2]:
features = pd.read_csv('blogfeatures.csv').sample(frac=1.0)
del features['Unnamed: 0']
del features['id']
lang = list(features.language.unique())

In [3]:
y = features['language'].values.reshape(-1, 1).ravel()
X = features[features.columns[~features.columns.str.contains('language')]]
X.head()

print(np.shape(y))
print(np.shape(X))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=39)  

(13087,)
(13087, 472)


<a id="seca"></a>

__Create Function for Comparing Models__

In [4]:
cols = ['name','time','total','precision','recall','f1']

model_set = pd.DataFrame(columns=cols)
models_stored = []
pattern = "%.2f"

In [5]:
def run_model(model,name):
    global model_set
    m = model
    m.fit(X_train, y_train)
    start = time.time()

    total_score = m.score(X_test,y_test)
    pscore = [pattern % i for i in list(metrics.precision_score(y_test, m.predict(X_test),labels=lang,average=None))]
    rscore = [pattern % i for i in list(metrics.recall_score(y_test, m.predict(X_test),labels=lang,average=None))]
    fscore = [pattern % i for i in list(metrics.f1_score(y_test, m.predict(X_test),labels=lang,average=None))]
    end = time.time()
    t= pattern % (end - start)

    r = dict(zip(cols,[name,t,total_score,pscore,rscore,fscore]))
    print('Total Score is: {}\n'.format(total_score))
    print(classification_report(y_test, m.predict(X_test)))
    
    model_set = model_set.append(r,ignore_index=True)
    return r,m

### <span style="color:gray">A. LR - Ordinary Least Squares _(not used)_</span>  <a href='#top'>(top)</a>

> Target is discrete so this model may not be appropriate <br>Many features are binary so model may not be appropriate

<a id="secb"></a>

### <span style="color:darkred">B. LR - Logistic Regression</span>  <a href='#top'>(top)</a>

> Target is binary so logistic regression will operate on probabilities

In [6]:
%%time
lreg_data,lreg = run_model(linear_model.LogisticRegression(),'Logistic Regression')

Total Score is: 0.7515274949083504

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00         8
           Japanese       0.77      0.94      0.84      1318
             Korean       0.20      0.02      0.04       152
Traditional Chinese       0.70      0.49      0.57       486

        avg / total       0.70      0.75      0.71      1964

CPU times: user 3.61 s, sys: 44.3 ms, total: 3.66 s
Wall time: 3.7 s


<a id="secb1"></a>

### <span style="color:gray">C. Lasso _(not used)_</span>  <a href='#top'>(top)</a>

<a id="secb2"></a>

### <span style="color:gray">D. Ridge _(not used)_</span>  <a href='#top'>(top)</a>

_Lasso and Ridge are not good predictors so should I just be using them for parameter manipulation?_

<a id="sece"></a>

### <span style="color:darkred">E. K Nearest Neighbors</span>  <a href='#top'>(top)</a>

> Can handle discrete values for target <br>Quantitative values are limited (not continuous) and might be problematic for nearest neighbors

In [7]:
%%time
neighbors_data,neighbors = run_model(KNeighborsClassifier(n_neighbors=5),'K Nearest Neighbor')

Total Score is: 0.6720977596741344

                     precision    recall  f1-score   support

            English       0.50      0.12      0.20         8
           Japanese       0.70      0.93      0.79      1318
             Korean       0.08      0.01      0.01       152
Traditional Chinese       0.49      0.20      0.29       486

        avg / total       0.60      0.67      0.61      1964

CPU times: user 3.16 s, sys: 17 ms, total: 3.17 s
Wall time: 3.18 s


<a id="secf"></a>

### <span style="color:darkred">F. Naive Bayes - Bernoulli</span>  <a href='#top'>(top)</a>

> Should be best for boolean classification but not for multiple discrete values (multinomial should be better)

In [8]:
%%time
bnb_data,bnb = run_model(BernoulliNB(),'Naive Bayes - Bernoulli')

Total Score is: 0.6232179226069247

                     precision    recall  f1-score   support

            English       0.01      0.12      0.02         8
           Japanese       0.74      0.78      0.76      1318
             Korean       0.17      0.14      0.15       152
Traditional Chinese       0.46      0.37      0.41       486

        avg / total       0.63      0.62      0.62      1964

CPU times: user 197 ms, sys: 63 ms, total: 260 ms
Wall time: 264 ms


<a id="secg"></a>

### <span style="color:darkred">G. Decision Tree</span>  <a href='#top'>(top)</a>

> Visualizes most important features by hierarchy <br>Longer processing time

In [9]:
%%time
dt_data,dt = run_model(tree.DecisionTreeClassifier(criterion='entropy',max_depth=6),'Decision Tree')

Total Score is: 0.7128309572301426

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00         8
           Japanese       0.72      0.94      0.82      1318
             Korean       0.00      0.00      0.00       152
Traditional Chinese       0.64      0.33      0.43       486

        avg / total       0.64      0.71      0.66      1964

CPU times: user 246 ms, sys: 9.21 ms, total: 256 ms
Wall time: 257 ms


/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [10]:
# Render tree.
dot_data = tree.export_graphviz(
    dt, out_file=None,
    feature_names=X.columns,
    class_names=lang,
    filled=True
)

graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

graph.write_png('decision_tree.png')

True

In [32]:
dimportance = list(zip(X.columns,dt.feature_importances_))
dimportance = dict(dimportance)

In [35]:
a1_sorted_keys = sorted(dimportance, key=dimportance.get, reverse=True)
p = []
for r in a1_sorted_keys:
    if dimportance[r] != 0:
        p.append(r)
        print(r, dimportance[r])
        
print(p)

sc 0.35570559754690545
pos2_NN-MD 0.13974544337906813
wc 0.08891427371866158
pos2_DT-JJ 0.07914412860841938
pos2_PRP-VBD 0.04644976054445818
pos2_NN-PRP 0.023614825593850714
pos2_PRP-RB 0.02213387334385531
pos2_MD-VB 0.022005434187654892
pos2_VB-PRP 0.02002761061047981
pos2_PRP-DT 0.013241864535248572
pos2_WRB-NN 0.011973156954026223
pos2_RB-JJ 0.010855596257393648
pos2_RB-MD 0.010433172272511582
pos2_IN-NNP 0.010072396940178526
pos2_JJ-PRP 0.009621070289970321
pos2_JJR-CC 0.009572150069413747
pos2_DT-RBS 0.00952086499973514
pos2_NN-IN 0.008502144092168715
pos2_VBG-NNP 0.008261530131257131
pos2_NNP-NNP 0.007884609861541294
pos2_CC-PRP$ 0.0075910084952799835
pos2_NNS-JJ 0.007482403475987729
pos2_JJ-VBP 0.0071029105478891364
pos2_VBP-VB 0.0070325757717591645
pos2_WRB-MD 0.007032534754542569
pos2_NNP-IN 0.006872619526808276
pos2_VB-NN 0.006346244443534983
pos2_NNP-JJ 0.006035819939534535
pos2_VBZ-JJ 0.005827202617913149
pos2_NNP-POS 0.005556653519716059
pos2_VBD-TO 0.005352101972706334
po

_Good visualization of important features and presentation of entropy weighting_

<a id="sech"></a>

### <span style="color:darkred">H. Random Forest</span>  <a href='#top'>(top)</a>

> Runs decision tree multiple times for best output <br>Longest processing time

In [11]:
%%time
rf_data,rf = run_model(ensemble.RandomForestClassifier(n_estimators=20),'Random Forest')

Total Score is: 0.7194501018329938

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00         8
           Japanese       0.72      0.95      0.82      1318
             Korean       1.00      0.01      0.03       152
Traditional Chinese       0.68      0.31      0.43       486

        avg / total       0.73      0.72      0.66      1964

CPU times: user 808 ms, sys: 25.9 ms, total: 833 ms
Wall time: 840 ms


/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.

/Users/ryan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [12]:
cvs = cross_val_score(rf, X_test, y_test, cv=5)
print(cvs)
scoreH = cvs.sum()/len(cvs)

[0.69873418 0.68527919 0.70483461 0.69565217 0.70588235]


In [13]:
rf.score(X_train,y_train)
print(scoreH)

0.6980764994969266


<a id="sec2"></a>

# <span style="color:darkblue">2. Model Comparison</span>  <a href='#top'>(top)</a>

In [42]:
model_set.columns = ['name','time','total','prec: | JA | CH | EN | KO |','rec: | JA | CH | EN | KO |','f1: | JA | CH | EN | KO |']
model_set

,name,time,total,prec: | JA | CH | EN | KO |,rec: | JA | CH | EN | KO |,f1: | JA | CH | EN | KO |
0,Logistic Regression,0.03,0.751527,"[0.77, 0.20, 0.70, 0.00]","[0.94, 0.02, 0.49, 0.00]","[0.84, 0.04, 0.57, 0.00]"
1,K Nearest Neighbor,2.39,0.672098,"[0.70, 0.08, 0.49, 0.50]","[0.93, 0.01, 0.20, 0.12]","[0.79, 0.01, 0.29, 0.20]"
2,Naive Bayes - Bernoulli,0.07,0.623218,"[0.74, 0.17, 0.46, 0.01]","[0.78, 0.14, 0.37, 0.12]","[0.76, 0.15, 0.41, 0.02]"
3,Decision Tree,0.02,0.712831,"[0.72, 0.00, 0.64, 0.00]","[0.94, 0.00, 0.33, 0.00]","[0.82, 0.00, 0.43, 0.00]"
4,Random Forest,0.07,0.719450,"[0.72, 1.00, 0.68, 0.00]","[0.95, 0.01, 0.31, 0.00]","[0.82, 0.03, 0.43, 0.00]"


-----